In [1]:
from melbanks import LogMelFilterBanks
import torch
import torchaudio

In [2]:
signal, sr = torchaudio.load("/home/darinka/Downloads/My Little Black Angel Death In June.wav")

In [3]:
melspec = torchaudio.transforms.MelSpectrogram(
    hop_length=160,
    n_mels=80
)(signal)
logmelbanks = LogMelFilterBanks()(signal)

assert torch.log(melspec + 1e-6).shape == logmelbanks.shape
assert torch.allclose(torch.log(melspec + 1e-6), logmelbanks)

TypeError: stft() got an unexpected keyword argument 'win_lenght'. Did you mean 'win_length'?